In [2]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
#Create a sqlite database with hop_team as a table from the hop_teaming dataset.
db = sqlite3.connect('data/hop_team.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv',
                       chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('hop_team', db, if_exists = 'append', index = False)
    
#db.execute('CREATE INDEX from_npi ON hop_team(from_npi)')

db.close()

0it [00:00, ?it/s]

In [4]:
#index the hop_team table
db = sqlite3.connect('data/hop_team.sqlite')
db.execute('CREATE INDEX from_npi ON hop_team(from_npi)')
db.close()

In [32]:
#SQL query to filter down the table and save to a dataframe.
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM hop_team WHERE transaction_count >= 50 AND average_day_wait < 50"

hop_team_sqlite = pd.read_sql(query, db)

db.close()

In [9]:
#Create a cbsa table in the hop_team database.
db = sqlite3.connect('data/hop_team.sqlite')
cbsa = pd.read_excel('data/ZIP_CBSA_122021.xlsx')
cbsa.columns = [x.lower().replace(' ', '_') for x in cbsa.columns]

cbsa.to_sql('cbsa', db, if_exists = 'append', index = False)


db.close()

In [29]:
#filter for Nashville cbsa's. 
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM cbsa WHERE cbsa = 34980"

cbsa_sqlite = pd.read_sql(query, db)

db.close()
cbsa_sqlite.head()

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37190,34980,WOODBURY,TN,0.990999,0.991435,1.0,0.991154
1,37046,34980,COLLEGE GROVE,TN,0.992654,1.000000,1.0,0.992839
2,37077,34980,HENDERSONVILLE,TN,1.000000,1.000000,1.0,1.000000
3,37101,34980,MC EWEN,TN,0.002121,0.000000,0.0,0.002000
4,37026,34980,BRADYVILLE,TN,0.967974,1.000000,1.0,0.969112


In [3]:
#
db = sqlite3.connect('data/hop_team.sqlite')
taxonomy = pd.DataFrame()
for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv', chunksize = 10000)):
    taxonomy_switch = pd.melt(chunk, id_vars=['NPI'], value_vars = ['Healthcare Provider Primary Taxonomy Switch_1',
                                            'Healthcare Provider Primary Taxonomy Switch_2',
                                            'Healthcare Provider Primary Taxonomy Switch_3',
                                            'Healthcare Provider Primary Taxonomy Switch_4',
                                            'Healthcare Provider Primary Taxonomy Switch_5',
                                            'Healthcare Provider Primary Taxonomy Switch_6',
                                            'Healthcare Provider Primary Taxonomy Switch_7',
                                            'Healthcare Provider Primary Taxonomy Switch_8',
                                            'Healthcare Provider Primary Taxonomy Switch_9',
                                            'Healthcare Provider Primary Taxonomy Switch_10',
                                            'Healthcare Provider Primary Taxonomy Switch_11',
                                            'Healthcare Provider Primary Taxonomy Switch_12',
                                            'Healthcare Provider Primary Taxonomy Switch_13',
                                            'Healthcare Provider Primary Taxonomy Switch_14',
                                            'Healthcare Provider Primary Taxonomy Switch_15'])
    taxonomy_switch = taxonomy_switch.rename(columns={'value':'taxonomy_switch'})
    taxonomy_code=pd.melt(chunk, id_vars=['NPI'], value_vars = ['Healthcare Provider Taxonomy Code_1',
                                           'Healthcare Provider Taxonomy Code_2',
                                           'Healthcare Provider Taxonomy Code_3',
                                           'Healthcare Provider Taxonomy Code_4',
                                           'Healthcare Provider Taxonomy Code_5',
                                           'Healthcare Provider Taxonomy Code_6',
                                           'Healthcare Provider Taxonomy Code_7',
                                           'Healthcare Provider Taxonomy Code_8',
                                           'Healthcare Provider Taxonomy Code_9',
                                           'Healthcare Provider Taxonomy Code_10',
                                           'Healthcare Provider Taxonomy Code_11',
                                           'Healthcare Provider Taxonomy Code_12',
                                           'Healthcare Provider Taxonomy Code_13',
                                           'Healthcare Provider Taxonomy Code_14',
                                           'Healthcare Provider Taxonomy Code_15'])
    taxonomy_code = taxonomy_code.rename(columns={'value':'taxonomy_code'})
    taxonomy_code['code_number'] = taxonomy_code['variable'].str.extract('(\d\d?)')
    taxonomy_switch['code_number'] =  taxonomy_switch['variable'].str.extract('(\d\d?)')
    tax_temp = pd.merge(taxonomy_code,taxonomy_switch, on=['NPI','code_number'])[['NPI','taxonomy_switch','taxonomy_code']]
    chunk = taxonomy.append(tax_temp)
    chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)

0it [00:00, ?it/s]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,34,72,76,80,84,87,89,90,91,93,94,95,97,98,99,101,102,103,105,106,163,171,175,178,179,181,182,183,185,186,189,190,191,193,194,197,198,201,205,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,76,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,155,163,171,175,178,179,183,186,187,189,190,191,193,194,195,197,198,199,201,202,205,206,209,211,213,218,221,223,225,227,229,233,237,241,245,247,249,253,257,261,265,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: Dty

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,27,32,35,68,76,80,84,85,88,89,92,93,95,96,97,98,99,100,101,102,103,106,151,159,163,167,169,170,171,173,174,175,177,178,179,181,182,183,185,186,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,35,68,72,73,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,102,103,105,106,155,163,171,174,177,178,181,182,183,185,186,187,189,190,191,193,194,195,197,198,201,202,203,205,206,207,209,210,211,213,214,215,217,218,219,221,222,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarnin

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,34,76,80,81,84,85,88,89,92,93,95,96,97,98,99,100,101,102,103,104,105,106,179,183,185,186,189,190,193,195,197,198,199,201,202,203,205,206,209,210,213,214,217,218,221,222,225,226,227,229,230,231,233,234,235,237,238,241,242,245,246,247,249,250,253,254,255,257,258,259,261,262,265,266,269,271,273,274,277,278,281,282,285,286,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,35,40,72,76,77,80,81,84,85,88,89,92,93,96,97,100,101,104,105,179,181,182,183,185,186,187,189,190,191,193,194,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPy

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,27,34,40,46,76,80,84,88,92,93,96,97,100,101,155,159,163,167,169,171,173,174,175,177,178,179,181,182,183,185,186,187,189,190,191,193,194,195,197,198,201,202,203,205,206,209,210,211,213,214,217,218,221,222,225,226,229,230,231,233,234,235,237,238,241,242,245,246,247,249,250) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,76,77,80,81,84,85,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,105,106,166,170,174,178,179,182,183,185,186,187,189,190,191,193,194,195,197,198,199,201,203,205,207,209,211,213,215,217,219,221,223,225,227,229,231,233,235,237,239,241,243,245,247,249,251,253,255,257,259,261,263,265,267,269,271,273,275,277,279,281,283,285,287,289,291,293,295,297,299,301,325,326,327,328) have mixed 

...

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,60,64,68,72,73,76,77,80,81,84,85,87,89,90,91,93,94,95,98,99,102,103,106,115,122,125,126,127,129,130,133,134,137,138,141,142,145,146,149,153,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (24,32,40,60,64,65,69,73,77,87,90,91,94,95,98,99,102,103,106,115,119,121,122,123,125,126,127,129,130,131,133,134,137,141,322,323,324,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (24,40,64,65,68,69,72,73,77,95,98,99,102,103,106,115,119,122,125,126,127,129,130,131,133,134,137,138,141,142,145,146,324,325

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (32,64,68,69,72,73,76,77,81,84,85,89,92,93,97,99,101,102,103,105,106,115,118,121,122,125,126,129,130,133,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,64,68,69,72,73,77,103,106,115,117,119,121,122,123,125,126,129,130,133,134,137,138,141,142,143,145,146,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,60,64,68,69,72,73,76,77,80,81,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,106,115,117,118,119,121,122,320,321,322,323,324,325,326,327) have mixed types.Specify dtype 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,40,60,61,65,69,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106,115,118,121,122,125,126,127,129,130,134,319,320,321,322,323,324,325,326) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,40,60,64,68,69,73,75,77,78,79,81,82,83,85,86,87,88,89,90,91,94,95,98,99,101,102,103,106,115,117,118,119,121,122,123,125,126,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,40,64,65,68,69,72,73,75,77,78,79,81,82,83,84,85,86,87,89,90,91,93,94,95,98,99,102,103,106,115,117,118,321,322,323,324,325

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (24,32,64,65,68,69,72,73,76,77,81,85,89,93,97,99,101,102,103,105,106,115,117,118,119,121,122,123,125,126,127,129,130,133,134,137,138,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,40,64,68,69,73,77,81,85,87,89,90,91,93,94,95,97,98,99,101,102,103,105,106,115,118,121,125,129,133,137,141,145,149,153,157,161,165,169,173,177,181,185,189,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,40,64,68,69,72,73,77,81,85,89,91,94,95,98,99,102,103,106,115,117,118

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,40,60,64,65,68,69,73,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106,115,118,119,121,122,321,322,323,324,325,326,327) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (24,32,60,64,68,69,73,77,81,85,89,93,97,103,106,115,117,118,121,122,125,126,129,130,131,133,134,320,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,39,60,61,64,65,68,69,72,73,76,77,79,80,81,82,83,85,86,87,89,90,91,93,94,95,97,98,99,101,102,103,105,106,115,118,119,121,122,125,312,319,320,321,322,323,324,325,326,

In [38]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM taxonomy WHERE taxonomy_switch = 'Y'"

taxonomy_sqlite = pd.read_sql(query, db)

db.close()

taxonomy_sqlite.head()

,NPI,taxonomy_switch,taxonomy_code
0,1679576722,Y,207X00000X
1,1588667638,Y,207RC0000X
2,1497758544,Y,251G00000X
3,1023011178,Y,251G00000X
4,1841293990,Y,231H00000X


In [43]:
db = sqlite3.connect('data/hop_team.sqlite')
nucc = pd.read_csv('data/nucc_taxonomy_220.csv')
nucc.columns = [x.lower().replace(' ', '_') for x in nucc.columns]

nucc.to_sql('nucc', db, if_exists = 'append', index = False)

db.close()

In [44]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM nucc "

nucc_sqlite = pd.read_sql(query, db)

db.close()

nucc_sqlite.head()

,code,grouping,classification,specialization,definition,notes,display_name,section
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,None,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,None,Clinical & Laboratory Immunology (Allergy & Im...,Individual


In [45]:
nucc_sqlite.head(50)

,code,grouping,classification,specialization,definition,notes,display_name,section
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,None,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,None,Clinical & Laboratory Immunology (Allergy & Im...,Individual
5,207L00000X,Allopathic & Osteopathic Physicians,Anesthesiology,None,An anesthesiologist is trained to provide pain...,"Source: American Board of Medical Specialties,...",Anesthesiology Physician,Individual
6,207LA0401X,Allopathic & Osteopathic Physicians,Anesthesiology,Addiction Medicine,An anesthesiologist who specializes in the dia...,"Source: National Uniform Claim Committee, 2009...",Addiction Medicine (Anesthesiology) Physician,Individual
7,207LC0200X,Allopathic & Osteopathic Physicians,Anesthesiology,Critical Care Medicine,"An anesthesiologist, who specializes in critic...","Source: American Board of Medical Specialties,...",Critical Care Medicine (Anesthesiology) Physician,Individual
8,207LH0002X,Allopathic & Osteopathic Physicians,Anesthesiology,Hospice and Palliative Medicine,An anesthesiologist with special knowledge and...,Source: American Academy of Hospice and Pallia...,Hospice and Palliative Medicine (Anesthesiolog...,Individual
9,207LP2900X,Allopathic & Osteopathic Physicians,Anesthesiology,Pain Medicine,An anesthesiologist who provides a high level ...,"Source: American Board of Medical Specialties,...",Pain Medicine (Anesthesiology) Physician,Individual


Need to pull in the NUCC data set so the taxonomy_code is not an ambigious number. SQL join nucc with taxonomy on taxonomy code for taxonomy and code for nucc. I want to fill in specialization with classification for places where specialization is none. 

In [ ]:
query = 'PRAGMA table_info ("npi")'
with sqlite3.connect('data/hop_team.sqlite') as db:
    tables = pd.read_sql(query,db)

In [37]:
hop_team_sqlite.shape

(34176938, 6)

In [35]:
cbsa_sqlite.shape

(471, 8)

In [40]:
taxonomy_sqlite.shape

(6929563, 3)